
# PROBLEM B3
Build a CNN based classifier for Rock-Paper-Scissors dataset.
Your input layer should accept 150x150 with 3 bytes color as the input shape.
This is unlabeled data, use ImageDataGenerator to automatically label it.
Don't use lambda layers in your model.

The dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).

Desired accuracy AND validation_accuracy > 83%


Prepare the imports

In [1]:
!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [2]:
!pip install pandas==2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


In [3]:
!pip install Pillow==10.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [4]:
!pip install scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [5]:
!pip install tensorflow==2.13.0

In [6]:
!pip install tensorflow-datasets==4.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.9.3
    Uninstalling tensorflow-datasets-4.9.3:
      Successfully uninstalled tensorflow-datasets-4.9.3


Imports

In [1]:
import numpy as np
import pandas as pd
import PIL as pil
import scipy as sc
import tensorflow as tf
import tensorflow_datasets as tfds

Check Version

In [2]:
print(np.__version__)
print(pd.__version__)
print(pil.__version__)
print(sc.__version__)
print(tf.__version__)
print(tfds.__version__)

1.24.3
2.0.3
10.0.0
1.10.1
2.13.0
4.9.2


Prepare the Data

In [4]:
data, info = tfds.load('rock_paper_scissors', split=['train[:80%]','train[-20%:]'],with_info=True, as_supervised=True)

In [5]:
train_data, test_data = data

In [27]:
train_x = []
train_y = []

test_x = []
test_y = []

In [28]:
for image, label in train_data:
  image = tf.image.resize(image,(150,150))
  train_x.append(image.numpy())
  train_y.append(label.numpy())

In [29]:
for image, label in test_data:
  image = tf.image.resize(image,(150,150))
  test_x.append(image.numpy())
  test_y.append(label.numpy())

In [30]:
train_x = np.array(train_x)
train_y = np.array(train_y)

test_x = np.array(test_x)
test_y = np.array(test_y)

In [31]:
train_x.shape

(2016, 150, 150, 3)

In [32]:
train_y.shape

(2016,)

Image Data Generator

In [33]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.,
                                                                shear_range=0.1,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True,
                                                                rotation_range=40,
                                                                fill_mode='nearest')
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)

In [34]:
train_gen = train_datagen.flow(train_x, train_y)
test_gen = test_datagen.flow(test_x, test_y)

Model Creation

In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(150,150,3)),
    tf.keras.layers.Conv2D(16,(3,3),1, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),1, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16,(3,3),1, activation='relu'),
    tf.keras.layers.Flatten(), # tried using GlobalAveragePooling2D instead of flatten, it does not yield good accuracy
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [48]:
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'rmsprop', metrics=['accuracy'])

In [49]:
model.fit(train_gen, validation_data=test_gen, epochs=10)

Epoch 1/10
63/63 [==============================] - 14s 189ms/step - loss: 1.1921 - accuracy: 0.3279 - val_loss: 1.0834 - val_accuracy: 0.3413
Epoch 2/10
63/63 [==============================] - 13s 203ms/step - loss: 1.0626 - accuracy: 0.4072 - val_loss: 0.8053 - val_accuracy: 0.7183
Epoch 3/10
63/63 [==============================] - 14s 227ms/step - loss: 0.8696 - accuracy: 0.6141 - val_loss: 0.5280 - val_accuracy: 0.7440
Epoch 4/10
63/63 [==============================] - 19s 299ms/step - loss: 0.7080 - accuracy: 0.6935 - val_loss: 0.3835 - val_accuracy: 0.9286
Epoch 5/10
63/63 [==============================] - 11s 182ms/step - loss: 0.6181 - accuracy: 0.7560 - val_loss: 0.4473 - val_accuracy: 0.7460
Epoch 6/10
63/63 [==============================] - 13s 202ms/step - loss: 0.5095 - accuracy: 0.7996 - val_loss: 0.4510 - val_accuracy: 0.6964
Epoch 7/10
63/63 [==============================] - 12s 187ms/step - loss: 0.4654 - accuracy: 0.8398 - val_loss: 0.1981 - val_accuracy: 0.9325